In [48]:
from z3 import *

# Define o tamanho do bit para a precisão dos inteiros
# Solicita o valor de n ao usuário
n = int(input("Insira o número de bits para a precisão (n): "))

# Definindo as variáveis de bit com o tamanho especificado pelo usuário
a = BitVec('a', n)
b = BitVec('b', n)
x = BitVec('x', n)
y = BitVec('y', n)
z = BitVec('z', n)  

# Estado inicial
initial_state = And(x == a, y == b, z == 0)

# Relação de transição:
# Se y for par, então x, y, z = x << 1, y >> 1, z
# Se y for ímpar, então x, y, z = x, y - 1, z + x
transition_even = And(y % 2 == 0, x == x << 1, y == y >> 1, z == z)
transition_odd = And(y % 2 != 0, x == x, y == y - 1, z == z + x)

# Transição para o próximo estado
next_state = Or(transition_even, transition_odd)

# Estado de erro - overflow nas operações
# Detectamos overflow verificando se os valores excedem o tamanho do bit n
overflow_shift = (x << 1).size() > n  # Operação de deslocamento à esquerda
overflow_sum = (z + x).size() > n  # Operação de soma

error_state = Or(overflow_shift, overflow_sum)


In [49]:
# Definindo o solver
solver = Solver()

# Base da indução: garantir que o invariante é verdadeiro no estado inicial
invariant = (x * y + z == a * b)
base_case = Implies(initial_state, invariant)
solver.add(Not(base_case))

# Verificando se o base_case é válido
if solver.check() == unsat:
    print("Base da indução mantida.")
else:
    print("Base da indução falhou:", solver.model())

# Passo da indução: verificar se o invariante continua válido após a transição
induction_step = Implies(And(invariant, next_state), invariant)
solver.push()
solver.add(Not(induction_step))

# Verificando o passo da indução
if solver.check() == unsat:
    print("Passo da indução mantido.")
else:
    print("Passo da indução falhou:", solver.model())
solver.pop()


Base da indução mantida.
Passo da indução mantido.


In [50]:
# Condição de restrição inicial
safe_initial_condition = And(initial_state, a < 2**(n//2), b < 2**(n//2))

# Garantir que não há estado de erro acessível a partir da condição inicial
secure = Implies(safe_initial_condition, Not(error_state))
solver.push()
solver.add(Not(secure))

# Verificando a segurança
if solver.check() == unsat:
    print("Propriedade de segurança mantida. Nenhum estado de erro é acessível.")
else:
    print("A propriedade de segurança falhou:", solver.model())
solver.pop()


Propriedade de segurança mantida. Nenhum estado de erro é acessível.
